In [5]:
from langchain_community.document_loaders import PyPDFLoader
from dotenv import load_dotenv

load_dotenv()
loader = PyPDFLoader("data/act.pdf")
docs = loader.load()

CHUNK_SIZE = 1000
CHUNKS_OVERLAP = 200
TOP_K = 5
MODEL_NAME = "gpt-3.5-turbo"
EMBEDDING_MODEL = "text-embedding-3-large"

In [2]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import anthropic

text_splitter = RecursiveCharacterTextSplitter(chunk_size = CHUNK_SIZE, chunk_overlap = CHUNKS_OVERLAP)
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(docs, OpenAIEmbeddings(model=EMBEDDING_MODEL))
retriever = vector.as_retriever(search_kwargs={"k": TOP_K})


In [18]:
def get_related_docs(query):
    related_docs = retriever.get_relevant_documents(query)
    return related_docs

def get_page_content(related_docs):
    page_content = ""
    for i, doc in enumerate(related_docs):
        page_content += f"START OF CONTEXT PART {i+1}\n\n{doc.page_content}\n\nEND OF CONTEXT PART {i+1}\n\n"
    return page_content

def create_query(query):
    related_docs = get_related_docs(query)
    page_content = get_page_content(related_docs)
    final_query = f"""
    You are an expert banker with expert knowledge on complience and regulations. This question is asked from you.

    Question: {query}

    You are given relevent sections from the legal document as context. You need to provide the answer to the question based on the given context only.

    Context: {page_content}

    Give your answer only based on the context given above. If can't find the answer in the given context, say "I can't find the answer in the given context".

"""
    return final_query

def ask_claud(query):
    response = anthropic.Anthropic().messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=512,
        messages=[
            {"role": "user", "content": create_query(query)}
        ]
    )
    return(response.content[0].text)




In [21]:
from flask import Flask, jsonify, request, json
from flask_cors import CORS

app = Flask(__name__)
CORS(app)

@app.route('/status', methods=['GET'])
def get_status():
    #return status ok
    return jsonify({'status': 'ok'}), 200


#add a POST route at /chat. Get the request body and echo the message back
@app.route('/chat', methods=['POST'])
def chat():
    data = json.loads(request.data)
    
    #get the last message from the user
    message = data['messages'][-1]['content']
    print(message)
    response = ask_claud(message)
    #message = data['messages']
    return jsonify({'status': 'ok', 'response': response}), 200

if __name__ == '__main__':
    app.run(port=3002)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:3002
Press CTRL+C to quit
127.0.0.1 - - [09/Mar/2024 15:29:11] "GET /status HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2024 15:29:11] "GET /status HTTP/1.1" 200 -


When are closure orders issued for a licensed commercial bank incorporated outside Sri Lanka, according to the Banking Act?


127.0.0.1 - - [09/Mar/2024 15:30:26] "POST /chat HTTP/1.1" 200 -


Under the Banking Act, which section outlines the liability of the acquiring bank regarding foreign currency depositors of a defaulting bank?


127.0.0.1 - - [09/Mar/2024 15:40:35] "POST /chat HTTP/1.1" 200 -
